In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
# Import API key
from config import g_key

In [2]:
#Store Part 1 results into data frame
cities_results_df=pd.read_csv("../output_data/cities.csv")
cities_results_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lompoc,34.6391,-120.4579,53.60,82,1,16.11,US,1613182327
1,atuona,-9.8000,-139.0333,78.93,79,1,15.77,PF,1613182328
2,mataura,-46.1927,168.8643,73.99,52,85,1.99,NZ,1613182329
3,hermanus,-34.4187,19.2345,72.00,79,0,3.00,ZA,1613182330
4,palmer,42.1584,-72.3287,23.00,45,90,2.10,US,1613182148


In [12]:
#Humidity Heat maps
#Configure gmaps
gmaps.configure(api_key=g_key)
#get lat and lng and humidity
locations = cities_results_df[["Lat", "Lng"]]
rating = cities_results_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#Narrow Cities
#Temperature between 70 and 80
best_cities=cities_results_df[cities_results_df["Max Temp"]<=80]
best_cities=best_cities[best_cities["Max Temp"]>=70]
#Cloudiness <=10%
best_cities=best_cities[best_cities["Cloudiness"]<=10]
#Wind Speed <= 5 MPH
best_cities=best_cities[best_cities["Wind Speed"]<=5]
best_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,hermanus,-34.4187,19.2345,72.00,79,0,3.00,ZA,1613182330
10,busselton,-33.6500,115.3333,75.00,58,0,1.99,AU,1613182339
148,ahipara,-35.1667,173.1667,72.14,55,0,3.69,NZ,1613182514
193,pochutla,15.7432,-96.4661,78.01,74,6,3.83,MX,1613182571
213,acari,-15.4311,-74.6158,73.74,60,7,1.48,PE,1613182596
429,tambura,5.6000,27.4667,70.86,28,0,1.61,SS,1613182868
483,catio,11.2833,-15.2500,71.01,72,0,4.27,GW,1613182932
537,celestun,20.8667,-90.4000,75.47,83,2,3.89,MX,1613182999


In [9]:
#Find Hotels
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": "39.952583,-75.16522",
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
#Create empty lists for storing
cities=[]
countries=[]
hotel_names=[]
lats=[]
lngs=[]
# use iterrows to iterate through pandas dataframe
for index, row in best_cities.iterrows():
    #add location to params dict
    time.sleep(1)
    params['location'] = f'{best_cities.loc[index,"Lat"]},{best_cities.loc[index,"Lng"]}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        #print(f"Closest hotel to {best_cities.loc[index,'City']} is {results[0]['name']}.")
        hotel_names.append(results[0]['name'])
        cities.append(best_cities.loc[index, 'City'])
        countries.append(best_cities.loc[index, 'Country'])
        lats.append(best_cities.loc[index, 'Lat'])
        lngs.append(best_cities.loc[index, 'Lng'])
        #types_df.loc[index, 'name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotels_df=pd.DataFrame({"City": cities,
                        "Country": countries,
                        "Hotel Name": hotel_names,
                        "Lat": lats,
                        "Lng": lngs})
hotels_df.head()

,City,Country,Hotel Name,Lat,Lng
0,hermanus,ZA,Aloe House Guest Lodge,-34.4187,19.2345
1,busselton,AU,Pearls Place,-33.6500,115.3333
2,ahipara,NZ,Ahipara Holiday Park,-35.1667,173.1667
3,pochutla,MX,Hotel Posada San Jose,15.7432,-96.4661
4,acari,PE,HOSPEDAJE XIMENA,-15.4311,-74.6158


In [10]:
hotel_df=hotels_df

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [14]:
#Humidity Heat maps
#Configure gmaps
gmaps.configure(api_key=g_key)
#get lat and lng and humidity
locations = cities_results_df[["Lat", "Lng"]]
rating = cities_results_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max(rating),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)
# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(hotel_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))